In [ ]:
import numpy as np
import os
import pandas as pd

from pandasql import sqldf

In [ ]:
# Load patient records that is included in the study
df_patientdata = pd.read_csv('data/preprocessed/patient_data.csv', dtype={'stay_id': str, 'subject_id': str, 'acuity': str, 'disposition': str, 'complexity': str}, index_col='stay_id')
list_stayid = list(df_patientdata.index)
list_subjectid = list(df_patientdata['subject_id'])

df_patientdata

In [ ]:
# Load ed/edstays table
df_edstays = pd.read_csv('data/ed/edstays.csv', dtype={'stay_id': str, 'subject_id': str})
df_edstays['intime'] = pd.to_datetime(df_edstays['intime'])
df_edstays['outtime'] = pd.to_datetime(df_edstays['outtime'])
df_edstays = df_edstays[df_edstays['stay_id'].isin(list_stayid)]
df_edstays = df_edstays[['stay_id', 'subject_id', 'intime', 'outtime']]

In [ ]:
# Create event logs for 'Patient arrival' from ed/edstays table
df_logs_patient_arrival = df_edstays[['stay_id', 'intime']].rename(columns={'stay_id': 'case_id', 'intime': 'timestamp'})
df_logs_patient_arrival['activity_name'] = ['Patient arrival'] * len(df_logs_patient_arrival)
df_logs_patient_arrival = df_logs_patient_arrival[['case_id', 'activity_name', 'timestamp']]

df_logs_patient_arrival

In [ ]:
# Create event logs for 'Patient discharge' from ed/edstays table
df_logs_patient_discharge = df_edstays[['stay_id', 'outtime']].rename(columns={'stay_id': 'case_id', 'outtime': 'timestamp'})
df_logs_patient_discharge['activity_name'] = ['Patient discharge'] * len(df_logs_patient_discharge)
df_logs_patient_discharge = df_logs_patient_discharge[['case_id', 'activity_name', 'timestamp']]

df_logs_patient_discharge

In [ ]:
# Load ed/vitalsigns table
df_vitalsign = pd.read_csv('data/ed/vitalsign.csv', dtype={'stay_id': str, 'subject_id': str})
df_vitalsign['charttime'] = pd.to_datetime(df_vitalsign['charttime'])
df_vitalsign = df_vitalsign[df_vitalsign['stay_id'].isin(list_stayid)]

# Create event logs for 'Vital sign check' from ed/vitalsigns table
df_logs_vitalsign_check = df_vitalsign[['stay_id', 'charttime']].rename(columns={'stay_id': 'case_id', 'charttime': 'timestamp'})
df_logs_vitalsign_check['activity_name'] = ['Vital sign check'] * len(df_logs_vitalsign_check)
df_logs_vitalsign_check = df_logs_vitalsign_check[['case_id', 'activity_name', 'timestamp']]

df_logs_vitalsign_check

In [ ]:
# Load ed/pyxis table
df_pyxis = pd.read_csv('data/ed/pyxis.csv', dtype={'stay_id': str, 'subject_id': str})
df_pyxis['charttime'] = pd.to_datetime(df_pyxis['charttime'])
df_pyxis = df_pyxis[df_pyxis['stay_id'].isin(list_stayid)]

# Create event logs for 'Medicine dispensation' from ed/pyxis table
df_logs_medicine_dispensation = df_pyxis[['stay_id', 'charttime']].rename(columns={'stay_id': 'case_id', 'charttime': 'timestamp'})
df_logs_medicine_dispensation['activity_name'] = ['Medicine dispensation'] * len(df_logs_medicine_dispensation)
df_logs_medicine_dispensation = df_logs_medicine_dispensation[['case_id', 'activity_name', 'timestamp']]

df_logs_medicine_dispensation

In [ ]:
# Load hosp/emar table
df_emar = pd.read_csv('data/hosp/emar.csv', dtype={'subject_id': str})

df_emar

In [ ]:
# Create event logs for 'Medicine administration' from hosp/emar table
df_emar_logs = df_emar[df_emar['event_txt'].isin(['Started in Other Location', 'Administered in Other Location', 'Started', 'Administered'])]
df_emar_logs = df_emar_logs[df_emar_logs['subject_id'].isin(list_subjectid)]
df_emar_logs = df_emar_logs[['subject_id', 'charttime']]

query_medicine_administration = """
    SELECT *
    FROM df_edstays
    INNER JOIN df_emar_logs
    ON df_edstays.subject_id = df_emar_logs.subject_id
    WHERE df_emar_logs.charttime BETWEEN df_edstays.intime AND df_edstays.outtime
"""

df_logs_medicine_administration = sqldf(query_medicine_administration, globals())

df_logs_medicine_administration = df_logs_medicine_administration[['stay_id', 'charttime']].rename(columns={'stay_id': 'case_id', 'charttime': 'timestamp'})
df_logs_medicine_administration['activity_name'] = ['Medicine administration'] * len(df_logs_medicine_administration)
df_logs_medicine_administration = df_logs_medicine_administration[['case_id', 'activity_name', 'timestamp']]

df_logs_medicine_administration

In [ ]:
# Load hosp/poe table
df_poe = pd.read_csv('data/hosp/poe.csv', dtype={'subject_id': str})

df_poe

In [ ]:
# Create event logs for 'Imaging request' from hosp/poe table
df_poe_logs = df_poe[df_poe['order_subtype'].isin(['CT Scan', 'Ultrasound', 'General Xray', 'MRI'])]
df_poe_logs = df_poe_logs[df_poe_logs['subject_id'].isin(list_subjectid)]
df_poe_logs = df_poe_logs[['subject_id', 'ordertime']]

query_imaging_request = """
    SELECT *
    FROM df_edstays
    INNER JOIN df_poe_logs
    ON df_edstays.subject_id = df_poe_logs.subject_id
    WHERE df_poe_logs.ordertime BETWEEN df_edstays.intime AND df_edstays.outtime
"""

df_logs_imaging_request = sqldf(query_imaging_request, globals())

df_logs_imaging_request = df_logs_imaging_request[['stay_id', 'ordertime']].rename(columns={'stay_id': 'case_id', 'ordertime': 'timestamp'})
df_logs_imaging_request['activity_name'] = ['Imaging request'] * len(df_logs_imaging_request)
df_logs_imaging_request = df_logs_imaging_request[['case_id', 'activity_name', 'timestamp']]

df_logs_imaging_request

In [ ]:
# Load hosp/labevents table
BATCH_SIZE = 1000000
list_labevents_logs = []
for df_batch in pd.read_csv('data/hosp/labevents.csv', chunksize=BATCH_SIZE, low_memory=False):
    # Create event logs for 'Lab test' from hosp/labevents table
    df_batch['subject_id'] = df_batch['subject_id'].astype(str)
    df_labevents_logs_batch = df_batch[df_batch['subject_id'].isin(list_subjectid)]
    df_labevents_logs_batch = df_labevents_logs_batch[['subject_id', 'storetime']]
    list_labevents_logs.append(df_labevents_logs_batch)

df_labevents_logs = pd.concat(list_labevents_logs, ignore_index=True)

df_labevents_logs

In [ ]:
# Do in batches because table size is large
BATCH_SIZE = 100000
list_logs_lab_test = []
for idx in range(0, len(df_labevents_logs), BATCH_SIZE):
    df_batch = df_labevents_logs.iloc[idx:idx + BATCH_SIZE]

    query_lab_request = """
        SELECT *
        FROM df_edstays
        INNER JOIN df_batch
        ON df_edstays.subject_id = df_batch.subject_id
        WHERE df_batch.storetime BETWEEN df_edstays.intime AND df_edstays.outtime
    """

    df_batch_logs = sqldf(query_lab_request, globals())

    df_batch_logs = df_batch_logs[['stay_id', 'storetime']].rename(columns={'stay_id': 'case_id', 'storetime': 'timestamp'})
    df_batch_logs['activity_name'] = ['Lab test'] * len(df_batch_logs)
    df_batch_logs = df_batch_logs[['case_id', 'activity_name', 'timestamp']]
    list_logs_lab_test.append(df_batch_logs)

df_logs_lab_test = pd.concat(list_logs_lab_test, ignore_index=True)

df_logs_lab_test

In [ ]:
# Concatenate event logs of all ED activities
df_logs_ed = pd.concat([df_logs_patient_arrival, df_logs_vitalsign_check, df_logs_medicine_dispensation, df_logs_medicine_administration, df_logs_lab_test, df_logs_imaging_request, df_logs_patient_discharge])

df_logs_ed

In [ ]:
# Relabel and sort event logs
df_logs_ed['timestamp'] = pd.to_datetime(df_logs_ed['timestamp']).round('min')

dict_activity_idx = {
    'Patient arrival': 'A',
    'Vital sign check': 'B',
    'Medicine dispensation': 'C',
    'Medicine administration': 'D',
    'Lab test': 'E',
    'Imaging request': 'F',
    'Patient discharge': 'G'
}
for key in dict_activity_idx.keys():
    df_logs_ed['activity_name'] = np.where(df_logs_ed['activity_name'] == key, dict_activity_idx[key], df_logs_ed['activity_name'])

df_logs_ed = df_logs_ed.sort_values(['case_id', 'timestamp', 'activity_name'])

df_logs_ed

In [ ]:
# Save event logs that will be used for the study
outpath_edlogs = 'data/preprocessed'
if not os.path.exists(outpath_edlogs):
    os.makedirs(outpath_edlogs)

df_logs_ed.to_csv(f'{outpath_edlogs}/event_logs.csv', index=False)